In [1]:
# Run 1 time only
%pip install SpeechRecognition
%pip install gtts
%pip install pyttsx3
%pip install transformers
%pip install torch
%pip install "accelerate>=0.26.0"
%pip install pyaudio
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117
%pip install tf-keras

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu117
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import speech_recognition as srec
from gtts import gTTS
import pyttsx3 as pyt
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

In [3]:
print(torch.cuda.is_available())  # Should return True if CUDA is available
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))  # Name of the GPU
else:
    print("CUDA is not available")

True
NVIDIA GeForce GTX 1650


In [4]:
engine = pyt.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [5]:
# --- INDO MINSTRAL 7B MODEL ---

model = "indischepartij/MiaLatte-Indo-Mistral-7b"
tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


In [6]:
# --- STT ---

def perintah():
    mendengar = srec.Recognizer()
    with srec.Microphone() as source:
        print('Mendengarkan......')
        suara = mendengar.listen(source, phrase_time_limit=5)
        try:
            print('Diterima.....')
            dengar = mendengar.recognize_google(suara, language='id-ID')
            print(dengar)
        except:
            pass
        return dengar

In [7]:
# --- TTS ---

# def ngomong(self):
#     teks = (self)
#     bahasa = 'id'
#     namafile = 'Ngomong.mp3'
#     def reading():
#         suara = gTTS(text=teks, lang=bahasa, slow=False)
#         suara.save(namafile)
#         os.system(f'start {namafile}')
#     reading()

In [8]:
def ngomong(text):
    voices = engine.getProperty('voices')

    for voice in voices:
        if "MSTTS_V110_idID_Andika" in voice.id:
            engine.setProperty('voice', voice.id)
            break

    # Speak the text
    engine.say(text)
    
    # Wait until speaking is finished
    engine.runAndWait()

In [11]:
# --- RUNNING VOICE ASSISTANT ---
def run_va():
    # prompt from user
    Layanan = perintah()

    # messages = [{"role": "user", "content": Layanan}]
    

    

    # ngomong(Layanan)

    # print(Layanan)

    system_prompt= 'Jawab dalam bahasa indonesia.'

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "question", "content": Layanan}
        ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    prompt = tokenizer.apply_chat_template(text, tokenize=False, add_generation_prompt=True)

    # model_inputs = tokenizer([text], return_tensors="pt").to(device)  # Move to the same device as the model

    # input_ids = model_inputs.input_ids.to(device)  # Move input tensor to GPU

    # generated_ids = model.generate(
    #     input_ids,
    #     max_new_tokens=512,
    # )

    # generated_ids = [
    #     output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    # ]
    # response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    print(outputs[0]["generated_text"])

    print(outputs)
    ngomong(outputs)

run_va()

Mendengarkan......
Diterima.....
Siapa Presiden Indonesia saat ini


TemplateError: Conversation roles must alternate user/assistant/user/assistant/...